# Decorators


Decorators can be thought of as functions which modify the *functionality* of another function. They help to make your code shorter and more "Pythonic". 

To properly explain decorators we will slowly build up from functions. 



 So let's break down the steps:

## Functions Review

In [1]:
def func():
    return 1

In [2]:
func()

1

## Scope Review
Remember from the nested statements lecture that Python uses Scope to know what a label is referring to. For example:

In [20]:
s = 'Global Variable'

for i in range(12):
    r=4
    for j in range(4):
        p = 3
        
def check_for_locals():
    x=3
    for i in range(10):
        y=3
    print(locals())
    

Remember that Python functions create a new scope, meaning the function has its own namespace to find variable names when they are mentioned within the function. We can check for local variables and global variables with the <code>locals()</code> and <code>globals()</code> functions. For example:

In [21]:
check_for_locals()

{'x': 3, 'i': 9, 'y': 3}


In [22]:
print(globals().keys())

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 's', 'check_for_locals', '_i2', '_i3', '_i4', '_i5', '_i6', '_i7', '_i8', '_i9', '_i10', 'i', 'r', 'j', 'p', '_i11', '_i12', '_i13', '_i14', '_i15', '_i16', '_16', '_i17', '_i18', '_i19', '_i20', '_i21', '_i22'])


Here we get back a dictionary of all the global variables, many of them are predefined in Python. So let's go ahead and look at the keys:

In [23]:
print(globals().keys())

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 's', 'check_for_locals', '_i2', '_i3', '_i4', '_i5', '_i6', '_i7', '_i8', '_i9', '_i10', 'i', 'r', 'j', 'p', '_i11', '_i12', '_i13', '_i14', '_i15', '_i16', '_16', '_i17', '_i18', '_i19', '_i20', '_i21', '_i22', '_i23'])


Note how **s** is there, the Global Variable we defined as a string:

In [25]:
globals()['s']

'Global Variable'

Now let's run our function to check for local variables that might exist inside our function (there shouldn't be any)

In [26]:
check_for_locals()

{'x': 3, 'i': 9, 'y': 3}


Great! Now lets continue with building out the logic of what a decorator is. Remember that in Python **everything is an object**. That means functions are objects which can be assigned labels(variables) and passed into other functions. Lets start with some simple examples:

In [13]:
def hello(name='Jose'):
    return 'Hello '+name

In [14]:
hello()

'Hello Jose'

Assign another label to the function. Note that we are not using parentheses here because we are not calling the function **hello**, instead we are just passing a function object to the **greet** variable.

In [15]:
greet = hello

In [16]:
greet

<function __main__.hello(name='Jose')>

In [17]:
greet()

'Hello Jose'

So what happens when we delete the name **hello**?

In [18]:
del hello

In [19]:
hello()

NameError: name 'hello' is not defined

In [21]:
greet()

'Hello Jose'

Even though we deleted the name **hello**, the name **greet** * still points to* our original function object(i.e it had its own copy). It is important to know that functions are objects that can be passed to other objects!

## Functions within functions
Great! So we've seen how we can treat functions as objects, now let's see how we can define functions inside of other functions:

In [22]:
def hello(name='Jose'):
    print('The hello() function has been executed')
    
    def greet():
        return '\t This is inside the greet() function'
    
    def welcome():
        return "\t This is inside the welcome() function"
    
    print(greet())
    print(welcome())
    print("Now we are back inside the hello() function")

In [23]:
hello()

The hello() function has been executed
	 This is inside the greet() function
	 This is inside the welcome() function
Now we are back inside the hello() function


In [25]:
welcome() # welcome() function was locally defined in hello() therefore not available globally

NameError: name 'welcome' is not defined

Note how due to scope, the welcome() function is not defined outside of the hello() function. Now lets learn about returning functions from within functions:
## Returning Functions

In [34]:
def hello(name='Jose'):
    
    def greet():
        return '\t This is inside the greet() function'
    
    def welcome():
        return "\t This is inside the welcome() function"
    
    if name == 'Jose':
        return greet
    else:
        return welcome

Now let's see what function is returned if we set x = hello(), note how the empty parentheses means that name has been defined as Jose.

In [35]:
x = hello()

In [36]:
x

<function __main__.hello.<locals>.greet()>

Great! Now we can see how x is pointing to the greet function inside of the hello function.

In [38]:
print(x())

	 This is inside the greet() function


Let's take a quick look at the code again. 

In the <code>if</code>/<code>else</code> clause we are returning <code>greet</code> and <code>welcome</code>, not <code>greet()</code> and <code>welcome()</code>. 

This is because when you put a pair of parentheses after it, the function gets executed; whereas if you don’t put parentheses after it, then it can be passed around and can be assigned to other variables without executing it.

When we write <code>x = hello()</code>, hello() gets executed and because the name is Jose by default, the function <code>greet</code> is returned. If we change the statement to <code>x = hello(name = "Sam")</code> then the <code>welcome</code> function will be returned. We can also do <code>print(hello()())</code> which outputs *This is inside the greet() function*.

## Functions as Arguments
Now let's see how we can pass functions as arguments into other functions:

In [37]:
def hello():
    return 'Hi Jose!'

def other(func):
    print('Other code would go here')
    print(func())

In [38]:
other(hello)

Other code would go here
Hi Jose!


Great! Note how we can pass the functions as objects and then use them within other functions. Now we can get started with writing our first decorator:

## Creating a Decorator
- In the previous example we actually manually created a Decorator. Here we will modify it to make its use case clear:
- Arguments of the function to be wrapped are passed to wrap function ,inner one, therefore we must accept those arguments if any. 
- using `@decor_name` changes the function itself to it's wrapped version

In [108]:
def my_new_decorator(func):
    # basically wrapping with wrap_func() 
    # and wrapping the func() 
    def wrap_func():
        print("Code would be here, before executing the func")

        func()

        print("Code here will execute after the func()")
    # and returning it!
    return wrap_func

def func_needs_decorator():
    print("This function is in need of a Decorator")

In [109]:
# Reassign func_needs_decorator
func_needs_decorator = my_new_decorator(func_needs_decorator)

In [110]:
func_needs_decorator()

Code would be here, before executing the func
This function is in need of a Decorator
Code here will execute after the func()


So what just happened here? A decorator simply wrapped the function and modified its behavior and returns it as a wrapeed beautiful version . 
-Now let's understand how we can rewrite this code using the @ symbol, which is what Python uses for Decorators:

In [111]:
# if u basically want to off this decorator ,simply comment out the @decorator  line
@my_new_decorator
def func_needs_decorator():
    print("This function is in need of a Decorator")

In [112]:
func_needs_decorator()

Code would be here, before executing the func
This function is in need of a Decorator
Code here will execute after the func()


**Great! You've now built a Decorator manually and then saw how we can use the @ symbol in Python to automate this and clean our code. You'll run into Decorators a lot if you begin using Python for Web Development, such as Flask or Django!**

# Application of Decorator

- function declaration inside function cannt be done in c++ or java but in python we have decorators.

In [113]:
# point 2
def decorate(sum):
    def wrap(x,y):
        sum(x,y)
        print("sum is : " ,x+y)
    return wrap

In [114]:
@decorate
def sum(a,b):
    print("sum will we calculated by decorator")

In [115]:
sum(3,5)

sum will we calculated by decorator
sum is :  8
